<a href="https://colab.research.google.com/github/sirpps/youtube-splitter/blob/main/split_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Youtube Video Splitter
`En:` An code to trim youtube video or livestream easly, requeriment Pixeldrain api key, tap  [here](https://pixeldrain.com/user/api_keys)

`Id:` Sebuah kode untuk memotong video atau livestream youtube secara mudah, bahan yang diperlukan adalah Pixeldrain api key, klik [disini](https://pixeldrain.com/user/api_keys)

In [ ]:
#@title Install requirements
from IPython.display import clear_output as cls
import os
!pip install yt-dlp
!pip install requests
cls()
print("[installer] yt-dlp version:", os.popen("yt-dlp --version").read().strip())
print("[installer] yt-dlp has been installed")

In [ ]:
#@title Setup
#====================Making Path============
#@markdown ---
#@markdown `En:` Enter any youtube url (not yt short, cuz this function to trim long youtube vid)

#@markdown `Id:` Masukan link youtube (bukan yt short, karena kegunaan kode ini untuk memotong video panjang youtube)
youtube_url = "" #@param{type:"string"}
#@markdown ---
#@markdown `En:` Input with format `hh:mm:ss` if the video under 1 hour just put 00 in the hour selection

#@markdown `Id:` Masukan dengan format `jj:mm:dd` jika video kurang dari 1 jam isi pada bagian jj 00

#@markdown `En:` Note: h is hour, m is minute, s is second

#@markdown `Id:` Catatan: j itu jam, m itu menit, d itu detik
start_time = "00:00:00" #@param{type:"string"}
end_time = "00:00:00" #@param{type:"string"}
#@markdown ---
#@markdown Enter [pixeldrain](https://pixeldrain.com/user/api_keys) api key
pixeldrain_apikey = "" #@param{type:"string"}
print("[info] setup done")

In [ ]:
#@title Downloading youtube video
print("[youtube] Downloading yt video")
!yt-dlp -f 134 {youtube_url}
print("\n[youtube] Downloading yt audio")
!yt-dlp -f 140 --prefer-ffmpeg {youtube_url}
print("[youtube] Downloading done")
print("[youtube] Renaming file")
!mv *.mp4 vid_raw.mp4
!mv *.m4a aud_raw.m4a
print("[youtube] Done All")

In [ ]:
#@title Triming the video and audio
import time as t
print("[Trimer] Triming vid")
!ffmpeg -i *.mp4 -ss {start_time} -to {end_time} -c copy vid_out.mp4
t.sleep(3)
print("[Trimer] Triming aud")
!ffmpeg -i *.m4a -ss {start_time} -to {end_time} -c copy aud_out.m4a
#cls()
print("[Trimer] Done")
print("[Merge] Merging vid and aud")
!ffmpeg -i vid_out.mp4 -i aud_out.m4a -c:v copy -c:a aac -strict experimental output.mp4
print("[Merge] All Done")

In [ ]:
#@title Renaming video file
import subprocess
import json

vidtitel = subprocess.run(
    ['yt-dlp', '--dump-json', youtube_url],
    capture_output=True,
    text=True
)
try:
    titellama = json.loads(vidtitel.stdout)['title']
    titel = titellama.replace(" ", "-")
    titelbaru = f'{titel}.mp4'
    !mv output.mp4 {titelbaru}
except json.JSONDecodeError:
        print("An Error Occurred")

In [ ]:
#@title Uploading to [Pixeldrain](https://pixeldrain.com/)

upload = subprocess.run(
    ['curl', '-T', titelbaru, '-u', ':'+pixeldrain_apikey, 'https://pixeldrain.com/api/file/'],
    capture_output=True,
    text=True
)
#====================Debug============
#print("Output from curl:", upload.stdout)
#print("Error from curl:", upload.stderr)
#====================Making Path============
try:
    file_result_id = json.loads(upload.stdout)['id']
    print("[uploader] Upload done")
    print("file link: https://pixeldrain.com/u/" + file_result_id)
#    print("ID file:", file_result_id)
except json.JSONDecodeError:
    print("An error occured")